# L'image

In [50]:
img_path = 'DSC_3945.JPG'

![Image to test](DSC_3945.JPG "Château des Ducs de Bretagne")

# L'extraction des données géographiques

In [168]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif(filename):
    exif = Image.open(filename)._getexif()

    if exif is not None:
        for key, value in exif.items():
            name = TAGS.get(key, key)
            exif[name] = exif.pop(key)

        if 'GPSInfo' in exif:
            for key in exif['GPSInfo'].keys():
                name = GPSTAGS.get(key,key)
                exif['GPSInfo'][name] = exif['GPSInfo'].pop(key)

    return exif

exif = get_exif(img_path)
exif

{37121: b'\x01\x02\x03\x00',
 'ExposureBiasValue': (0, 3),
 'FlashPixVersion': b'0100',
 'MeteringMode': 5,
 'LightSource': 0,
 'Flash': 16,
 'FocalLength': (423, 100),
 'ColorSpace': 1,
 'ExifImageWidth': 5984,
 'ExifInteroperabilityOffset': 27792,
 'SceneCaptureType': 1,
 'SubjectDistanceRange': 0,
 36880: '+01:00',
 36881: '+01:00',
 36882: '+01:00',
 'SubsecTime': '600128',
 'SubsecTimeOriginal': '600128',
 'SubsecTimeDigitized': '600128',
 'ExifImageHeight': 3366,
 'Make': 'Sony',
 'Model': 'F5121',
 'Orientation': 1,
 'ExposureTime': (10, 6400),
 'XResolution': (72, 1),
 'YCbCrPositioning': 1,
 'FNumber': (20, 10),
 'YResolution': (72, 1),
 'GPSInfo': {27: b'ASCII\x00\x00\x00network',
  29: '2020:03:13',
  'GPSVersionID': b'\x02\x02\x00\x00',
  'GPSLatitudeRef': 'N',
  'GPSLatitude': ((47, 1), (12, 1), (55896, 1000)),
  'GPSLongitudeRef': 'W',
  'GPSLongitude': ((1, 1), (33, 1), (1940, 1000)),
  'GPSAltitudeRef': b'\x00',
  'GPSAltitude': (0, 1000),
  'GPSTimeStamp': ((9, 1), (17

### Visualiser les infos de géolocalisation

In [52]:
exif['GPSInfo']

{27: b'ASCII\x00\x00\x00network',
 29: '2020:03:13',
 'GPSVersionID': b'\x02\x02\x00\x00',
 'GPSLatitudeRef': 'N',
 'GPSLatitude': ((47, 1), (12, 1), (55896, 1000)),
 'GPSLongitudeRef': 'W',
 'GPSLongitude': ((1, 1), (33, 1), (1940, 1000)),
 'GPSAltitudeRef': b'\x00',
 'GPSAltitude': (0, 1000),
 'GPSTimeStamp': ((9, 1), (17, 1), (28000, 1000)),
 'GPSMapDatum': 'WGS-84',
 'GPSStatus': 'A'}

### Visualiser la date de la photo

In [53]:
exif['DateTime']

'2020:03:13 10:17:33'

### Visualiser la taille de l'image

In [54]:
exif['ExifImageWidth'], exif['ExifImageHeight']

(5984, 3366)

### Avoir les coordonnées au format DMS (degrés, minutes, secondes)

In [55]:
def get_coordinates(info):
    for key in ['Latitude', 'Longitude']:
        if 'GPS'+key in info and 'GPS'+key+'Ref' in info:
            e = info['GPS'+key]
            ref = info['GPS'+key+'Ref']
            info[key] = ( str(e[0][0]/e[0][1]) + '°' +
                          str(e[1][0]/e[1][1]) + '′' +
                          str(e[2][0]/e[2][1]) + '″ ' +
                          ref )

    if 'Latitude' in info and 'Longitude' in info:
        return [info['Latitude'], info['Longitude']]

coords = get_coordinates(exif['GPSInfo'])
coords

['47.0°12.0′55.896″ N', '1.0°33.0′1.94″ W']

### Avoir les données au format décimal

In [56]:
def get_decimal_coordinates(info):
    for key in ['Latitude', 'Longitude']:
        if 'GPS'+key in info and 'GPS'+key+'Ref' in info:
            e = info['GPS'+key]
            ref = info['GPS'+key+'Ref']
            info[key] = ( e[0][0]/e[0][1] +
                          e[1][0]/e[1][1] / 60 +
                          e[2][0]/e[2][1] / 3600
                        ) * (-1 if ref in ['S','W'] else 1)

    if 'Latitude' in info and 'Longitude' in info:
        return [info['Latitude'], info['Longitude']]

coords = get_decimal_coordinates(exif['GPSInfo'])
coords

[47.21552666666667, -1.550538888888889]

# L'affichage sur carte

In [57]:
# import libraries
import folium
import pandas as pd
 
# Make an empty map
m = folium.Map(location=[47, 1], tiles="OpenStreetMap", zoom_start=5)

m

In [58]:
# Make a data frame with dots to show on the map
data = pd.DataFrame({
                        'lat':[coords[0]],
                        'lon':[coords[1]],
                        'name':['Chateau des Ducs de Bretagne']
                    })

# I can add marker one by one on the map
for i in range(0,len(data)):
    folium.Marker([data.iloc[i]['lat'], data.iloc[i]['lon']], popup=data.iloc[i]['name']).add_to(m)
 
m

# Save it as html
#m.save('chateau_des_ducs_de_bretagne.html')

In [230]:
import matplotlib.pyplot as plt
import base64
import time
from tqdm import tqdm
from folium import IFrame
from folium.plugins import MarkerCluster


# Variable du redimentionnement
width = 300

def add_markers(m, data, width=300):
    
    print("Création de la carte ...")
    
    marker_cluster = MarkerCluster().add_to(m)

    # Balise html pour l'affichage Popup d'une image
    html = '<img src="data:image/png;base64,{0}">'.format

    time.sleep(0.1)
    for i in tqdm(range(len(data))):

        # Redimentionnement de l'image et enregistrement temporaire
        img = Image.open(data.iloc[i]['path'])
        height = int(img.size[1]*(width/img.size[0]))
        img = img.resize((width, height), Image.ANTIALIAS)
        if int(data.iloc[i]['orientation'])>1:
            img = img.transpose(Image.ROTATE_270)
            w, h = height, width
        else:
            w, h = width, height
        img.save('tmp.png')
        del img

        # Longitude / Latitude
        lat, lon = data.iloc[i]['lat'], data.iloc[i]['lon']

        # On ouvre l'image temporaire et on l'encode
        encoded = base64.b64encode(open('tmp.png', 'rb').read())

        # On en fait une Frame puis un Popup
        iframe = IFrame(html(encoded.decode('UTF-8'), w, h), width=w+20, height=h+20)
        popup = folium.Popup(iframe, max_width=2650)

        # On choisit l'icône
        icon = folium.Icon(icon="camera", prefix="fa")

        # On ajoute le Marker
        marker = folium.Marker([lat, lon], popup=popup, icon=icon)
        marker.add_to(marker_cluster)


    # On enregistre au format html
    print("Enregistrement de la carte ...")
    m.save('carte_geolocalisation_photos.html')

In [228]:
import os


def get_dataset(folder, data, main_call=True):
    
    if main_call:
        print("Récupération des données de géolocalisation ...")
    
    coords = []
    dates  = []
    times  = []
    paths  = []
    oriens = []
    
    for el_path in os.listdir(folder):
        
        el_path = os.path.join(folder, el_path)
        
        # Si c'est un fichier, on regarde s'il est sous un format d'image et on enregistre les informations
        if os.path.isfile(el_path) and el_path.split('.')[1] in ['png', 'jpg', 'jpeg', 'JPG']:
            
            exif = get_exif(el_path)
            
            if exif is not None:
            
                datetime = exif['DateTime'].split(' ')
                coord  = get_decimal_coordinates(exif['GPSInfo'])

                if coord is not None:

                    coords += [coord]
                    dates  += [datetime[0]]
                    times  += [datetime[1]]
                    paths  += [el_path]
                    oriens += [exif['Orientation']]
            
        # Si c'est un dossier, on cherche dedans de manière récursive
        elif os.path.isdir(el_path):
            
            sub_data = get_dataset(el_path, data, main_call=False)
            data     = data.append(sub_data)
            data     = data.drop_duplicates(ignore_index=True)
            
            
    tmp_data = pd.DataFrame({
                                'lat':[coord[0] for coord in coords],
                                'lon':[coord[1] for coord in coords],
                                'date':[date for date in dates],
                                'time':[time for time in times],
                                'path':[el_path for el_path in paths],
                                'orientation':[orien for orien in oriens]
                            })

    data = data.append(tmp_data)
    data = data.drop_duplicates(ignore_index=True)
            
    return data

In [229]:
# Info
start = time.time()
print('Début du script')

# On crée la base de données
data = pd.DataFrame({'lat': [], 'lon': [], 'date':[], 'time':[], 'path':[], 'orientation':[]})
data = get_dataset("C:\\Users\\alspe\\Pictures\\2020\\06-2020", data)

# On crée la carte
m = folium.Map(location=[47, 1], tiles="OpenStreetMap", zoom_start=5)

# On ajoute les emplacements
add_markers(m, data)

# Info
end = time.time()
print('Fin. Durée totale : ', int((end-start)//60), 'm', int((end-start)%60), 's')

Début du script
Récupération des données de géolocalisation ...
Création de la carte ...


  0%|          | 0/46 [00:00<?, ?it/s]


NameError: name 'weidth' is not defined